In [177]:
import os
import json
from pprint import pprint
import requests
import re

In [178]:
def toJson(htmlTable):
    table = {}
    table["rows"] = []
    rowStart = htmlTable.find("<tr")
    table["stylingHtml"] = htmlTable[:rowStart]
    rows = htmlTable[rowStart:htmlTable.find("</tbody>")].split("</tr>")
    for htmlRow in rows[:len(rows)-1]:
        row ={}
        row["cells"] = []
        cellStart = re.search("<t[hd]", htmlRow).start()
        row["stylingHtml"] = htmlRow[:cellStart]
        while re.search("<t[hd]", htmlRow[cellStart:]):
            cell = {}
            cellEnd = re.search("</t[hd]", htmlRow[cellStart:]).start() + 5 + cellStart
            cell["html"] = htmlRow[cellStart:cellEnd]
            row["cells"].append(cell)
            cellStart = cellEnd
        table["rows"].append(row)
    return json.dumps(table)
    

In [179]:
def getTables(html):
    tables = []
    tableBegin = 0
    splits = html.split('<table class="wikitable')
    # print("Len of splits: ")
    # print(len(splits))
    while html.find('<table class="wikitable', tableBegin) != -1:
        tableBegin = html.find('<table class="wikitable', tableBegin)
        # print("Table begin:" + str(tableBegin))
        tableEnd = html.find('</table>', tableBegin)
        # print("Table end:" + str(tableEnd))
        if tableEnd == -1:
            print("Could not find the end for the table.")
            return tables
        elif html.find('<table', tableBegin + 6, tableEnd) != -1:
            # print("Nested Begin:" + str(html.find('<table', tableBegin + 6, tableEnd)))
            print("We are currently skipping nested tables.")
            return tables
        else:
            tables.append(html[tableBegin:tableEnd + 8])
        tableBegin = tableEnd + 8
    return tables

In [180]:
def getRawHtml(table):
    pgId = table['pgId']
    url = 'https://en.wikipedia.org/?curid=' + str(pgId)
    response = requests.get(url)
    return response.text

In [181]:
input_file_path = '../data/100tables.json'
file = open(input_file_path, 'r')
for line in file:
    jsonTable = json.loads(line)
    print(jsonTable['pgId'])
    html = getRawHtml(jsonTable)
    htmlTables = getTables(html)
    pprint(toJson(htmlTables[0]))
    break
    

10000032
('{"rows": [{"cells": [{"html": "<th colspan=\\"2\\">Election</th>"}, {"html": '
 '"\\n<th>Member</th>"}, {"html": "\\n<th>Party\\n</th>"}], "stylingHtml": '
 '"<tr>\\n"}, {"cells": [{"html": "<td rowspan=\\"10\\" '
 'style=\\"background-color: #48A5EE\\">\\n</td>"}, {"html": "\\n<td><a '
 'href=\\"/wiki/Northern_Ireland_general_election,_1929\\" '
 'class=\\"mw-redirect\\" title=\\"Northern Ireland general election, '
 '1929\\">1929</a>\\n</td>"}, {"html": "\\n<td rowspan=\\"2\\"><a '
 'href=\\"/wiki/Robert_Crawford_(Antrim_politician)\\" title=\\"Robert '
 'Crawford (Antrim politician)\\">Robert Crawford</a>\\n</td>"}, {"html": '
 '"\\n<td rowspan=\\"10\\"><a href=\\"/wiki/Ulster_Unionist_Party\\" '
 'title=\\"Ulster Unionist Party\\">Ulster Unionist</a>\\n</td>"}], '
 '"stylingHtml": "\\n<tr>\\n"}, {"cells": [{"html": "<td><a '
 'href=\\"/wiki/Northern_Ireland_general_election,_1933\\" '
 'class=\\"mw-redirect\\" title=\\"Northern Ireland general election, '
 '1933\\">1933<